#### General Overview

1. Connect
1. Parse
1. Analyze (Semantics Analysis / Transformation Process in docs)
1. Rewrite (Rule System)
1. Plan (Optimizer)
1. Execute

<img src="./helpers/query-path-overview.png" alt="drawing" width="700"/>

#### Parser

##### Input / Output

- Input: Query string
- Output: Parse tree

##### Details:

1. Check for valid syntax (only fixes SQL grammar rules)
1. Generate a parse tree

##### Example:

<img src="./helpers/parse-tree-example.png" alt="drawing" width="700"/>

#### Analyzer

##### Input / Output

- Input: Parse tree
- Output: Query tree

##### Details

1. Query system catalogs
1. Interpret the referenced objects to columns, functions, tables, views, etc...
1. Interpret data types and data structures
1. This part open a transaction so it can already query system catalogs (that's why there is a separation from the previous one)

##### Example

<img src="./helpers/query-tree-example.png" alt="drawing" width="700"/>

#### Rewrite (Rule System)

##### Input / Output

- Input: Query tree
- Output: Query tree

**NOTE** - Pay attention, there is no data structure change

##### Details:


1. Idea - create a step between Analyzer and Planner that can manipulate the Query Tree object, one can think of it like a classic hook
1. Rules are stored in `pg_rules` which is a view wrapper around `pg_rewrite` table
1. Views are implemented with the rule system, when creating a view an empty relation is created with a `ON SELECT DO INSTEAD` rule that redirects to the view query

##### View Rewrite Example

With a view like:

```sql
CREATE VIEW employees_list AS 
SELECT e.id, e.name, d.name AS department 
FROM 
    employees AS e,
    departments AS d
WHERE e.department_id = d.id;
```

And a query like:

```sql
SELECT * FROM employees_list;
```
The rewrite system will process the query tree:

<img src="./helpers/rewrite-query-tree-view.png" alt="drawing" width="900"/>

##### Quick Demo
TODO - Try to do the demo before
Docker Compose:
- Add "-c", "debug_print_parse=on" to docker compose
- Issue a query
- Check for the appropriate log in the log file
On Debian
- Find hba.conf file
- Change the configuration from there
- Restart the server
- Issue a query
- Check for the appropriate log in the log file

#### Planner / Optimizer

##### Input / Output

- Input: Query Tree
- Output: Plan Tree